In [1]:
import requests as req
from datetime import datetime
import pandas as pd

from bs4 import BeautifulSoup as BS
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

from connection.var import ConnectionVar_forex, ConnectionVar_crypto, agent_desktop, parser, URL

In [2]:
crypto_columns = ['DATE','PRICE','PRICE_CAP', 'VOLUME_24','CIRCULATION', 'PERCENT_CHG']
data_columns = ['NAME','PRICE','PRICE_CAP', 'VOLUME_24','CIRCULATION', 'PERCENT_CHG']
tag_names = 'a'
forex_columns = ['DATE','LAST','CHG%','CHG','BID','ASK','HIGH','LOW','RaTING']

SIA = SentimentIntensityAnalyzer()

def get_html(url, header):
    #get data from sources
    #url for various sources
    connect = req.get(url, header)
    data = connect.content
    return data

def scrape_data_from_web(where, url_id, tag=None, className=None, id_name=None):

    date_as = datetime.strftime(datetime.now(), '%m-%d-%Y, %H:%M')
    scraped_data = []
    
    if where == 'crypto':
        
        conn = get_html(URL[url_id], header=agent_desktop)
        soup = BS(conn, parser)
        
        for k,v in ConnectionVar_crypto.items():
            scraped_data.append([x.get_text() for x in soup.find_all(tag, class_=v)])
        
        df = pd.DataFrame(index=scraped_data[0], columns=crypto_columns)

        df['DATE'] = date_as
        df['PRICE'] = scraped_data[1]
        df['PRICE_CAP'] = scraped_data[2]
        df['VOLUME_24'] = scraped_data[3]
        df['CIRCULATION'] = scraped_data[4]
        df['PERCENT_CHG'] = scraped_data[5]

        #df.to_sql(sqlite_table, sqlite_conn, if_exists='append')
        return df.head(10)
    
    elif where == 'forex':
        
        conn = get_html(URL[url_id], header=agent_desktop)
        soup = BS(conn, parser)
        
        
        scraped_data.append(([x.get_text() for x in soup.find_all('a',{'class':ConnectionVar_forex['a']})]))
        scraped_data.append([x.get_text() for x in soup.find_all('td',{'class':ConnectionVar_forex['td']})])
            
        #df = pd.DataFrame(index=scraped_data[0], columns=forex_columns)
        #df['DATE'] = date_as
        #df['LAST'] = scraped_data[1]
        #df['CHG%'] = scraped_data[2]
        #df['CHG'] = scraped_data[3]
        #df['BID'] = scraped_data[4]
        #df['ASK'] = scraped_data[5]
        #df['HIGH'] = scraped_data[6]
        #df['LOW'] = scraped_data[7]
        #df['RaTING'] = scraped_data[8]
        print(len(scraped_data[0]))
        print(len(scraped_data[1]))
        print(scraped_data)
        #df.to_sql(sqlite_table, sqlite_conn, if_exists='append')
        #return df.head(10)
        
    elif where == 'news'.lower():
        conn = get_html(URL[url_id], header=agent_desktop)
        soup = BS(conn, parser)
        
        scraped_data.append(([x.get_text() for x in soup.find_all(tag,class_=className)]))

        return scraped_data
        


In [3]:
newsHeadlines = scrape_data_from_web(where='news', tag='div', className='o-teaser__heading', url_id=3)
newsArticles = scrape_data_from_web(where='news', tag='p', className='o-teaser__standfirst', url_id=3)

In [4]:
newsHeadlinesPolarity = []

for polarityScores in range(len(newsHeadlines[0])):
    newsHeadlinesPolarity.append(SIA.polarity_scores(newsHeadlines[0][polarityScores]))   

In [5]:
newsDataFrame = pd.DataFrame(newsHeadlinesPolarity, columns=['neg','neu','pos'])

In [6]:
newsDataFrame['newsHeadline'] = newsHeadlines[0]


In [9]:
newsDataFrame

,neg,neu,pos,newsHeadline
0,0.000,0.781,0.219,Coronavirus latest: US disease control urges s...
1,0.000,1.000,0.000,"The globetrotting degree, grounded: executive ..."
2,0.143,0.667,0.190,Executive MBA alumni find advantage in tough j...
3,0.000,1.000,0.000,Global meets local as multinationals pair with...
4,0.000,1.000,0.000,FT Executive MBA ranking 2020 — methodology an...
5,0.000,0.659,0.341,Calculating where artificial intelligence can ...
6,0.000,1.000,0.000,What an executive MBA did for me
7,0.367,0.473,0.160,Why I went to business school to help child vi...
8,0.158,0.842,0.000,FT Executive MBA ranking 2020 analysis: demand...
9,0.000,1.000,0.000,How an executive MBA was the catalyst for a ph...


In [8]:
for x in range(len(newsDataFrame)):
    reqSend = req.post("http://127.0.0.1:5050/postnews/" + str(x) , data = {
        "newsArticle":newsDataFrame['newsHeadline'][x],
        "newsPolarityNeg":newsDataFrame['neg'][x],
        "newsPolarityPos":newsDataFrame['pos'][x],
        "newsPolarityNeu":newsDataFrame['neu'][x]

    })

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5050): Max retries exceeded with url: /postnews/0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022E89D95308>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))